# SequentialChain多链顺序执行 串行

## 源码分析

### 类定义和属性

In [ ]:
class SequentialChain(Chain):
    """Chain where the outputs of one chain feed directly into next."""

    chains: List[Chain]
    input_variables: List[str]
    output_variables: List[str]  #: :meta private:
    return_all: bool = False

- `chain`: 这是一个`Chain`对象的列表，表示要执行的链的集合。
- `input_variables`: 这个列表包含了所有需要作为输入提供给第一个链的变量名称。
- `output_variables`: 这个列表定义了整个 `SequentialChain` 的输出变量。通常，这些变量来自最后一个链的输出。
- `return_all`: 这是一个布尔值，如果设置为 True，则返回所有链的所有输出，否则只返回最后一个链的输出。


### 配置部分

In [ ]:
class Config:
    arbitrary_types_allowed = True
    extra = "forbid"

- `arbitrary_types_allowed`: 允许任意类型的属性。
- `extra`: 防止在类中定义额外的属性，这是一种防御机制，确保没有未定义的属性被传入。

### 属性方法

In [ ]:
@property
def input_keys(self) -> List[str]:
    """Return expected input keys to the chain."""
    return self.input_variables

@property
def output_keys(self) -> List[str]:
    """Return output key."""
    return self.output_variables

- `input_keys`: 返回 `SequentialChain` 所需的输入键，即 `input_variables`。
- `output_keys`: 返回 `SequentialChain` 的输出键，即 `output_variables`。

### 验证链的输入输出关系

In [ ]:
@root_validator(pre=True)
def validate_chains(cls, values: Dict) -> Dict:
    """Validate that the correct inputs exist for all chains."""
    chains = values["chains"]
    input_variables = values["input_variables"]
    memory_keys = list()

    if "memory" in values and values["memory"] is not None:
        """Validate that prompt input variables are consistent."""
        memory_keys = values["memory"].memory_variables
        if set(input_variables).intersection(set(memory_keys)):
            overlapping_keys = set(input_variables) & set(memory_keys)
            raise ValueError(
                f"The input key(s) {''.join(overlapping_keys)} are found "
                f"in the Memory keys ({memory_keys}) - please use input and "
                f"memory keys that don't overlap."
            )

    known_variables = set(input_variables + memory_keys)

    for chain in chains:
        missing_vars = set(chain.input_keys).difference(known_variables)
        if chain.memory:
            missing_vars = missing_vars.difference(chain.memory.memory_variables)

        if missing_vars:
            raise ValueError(
                f"Missing required input keys: {missing_vars}, "
                f"only had {known_variables}"
            )
        overlapping_keys = known_variables.intersection(chain.output_keys)
        if overlapping_keys:
            raise ValueError(
                f"Chain returned keys that already exist: {overlapping_keys}"
            )

        known_variables |= set(chain.output_keys)

    if "output_variables" not in values:
        if values.get("return_all", False):
            output_keys = known_variables.difference(input_variables)
        else:
            output_keys = chains[-1].output_keys
        values["output_variables"] = output_keys
    else:
        missing_vars = set(values["output_variables"]).difference(known_variables)
        if missing_vars:
            raise ValueError(
                f"Expected output variables that were not found: {missing_vars}."
            )

    return values

- `validate_chains`: 这是一个用于在链执行之前验证输入输出是否匹配的方法。它确保每个链的输入可以从已知的变量中获取，并且不会有重复的输出键。

### 主方法

*`_call`方法(同步执行)*

In [ ]:
def _call(
    self,
    inputs: Dict[str, str],
    run_manager: Optional[CallbackManagerForChainRun] = None,
) -> Dict[str, str]:
    known_values = inputs.copy()
    _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
    for i, chain in enumerate(self.chains):
        callbacks = _run_manager.get_child()
        outputs = chain(known_values, return_only_outputs=True, callbacks=callbacks)
        known_values.update(outputs)
    return {k: known_values[k] for k in self.output_variables}

- `_call`: 这是执行链的核心方法，它会依次执行所有链。每次执行完一个链后，它会将输出更新到 known_values 中，并将这些值传递给下一个链。

*`_acall`方法(异步执行)*

In [ ]:
async def _acall(
    self,
    inputs: Dict[str, Any],
    run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
) -> Dict[str, Any]:
    known_values = inputs.copy()
    _run_manager = run_manager or AsyncCallbackManagerForChainRun.get_noop_manager()
    callbacks = _run_manager.get_child()
    for i, chain in enumerate(self.chains):
        outputs = await chain.acall(
            known_values, return_only_outputs=True, callbacks=callbacks
        )
        known_values.update(outputs)
    return {k: known_values[k] for k in self.output_variables}

- `_acall`: 这是 `_call` 的异步版本，用于异步地执行链。异步调用适用于需要并发处理的场景。

### 总结
`SequentialChain` 类的核心功能是将多个链顺序连接起来，使一个链的输出直接成为下一个链的输入。通过引入 `validate_chains` 方法，它确保了链之间输入输出的匹配，以及避免了变量名的冲突。该类支持同步和异步的链式调用，提供了灵活的链式处理能力，非常适合复杂任务的分步执行。

## demo

In [44]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.sequential import SimpleSequentialChain, SequentialChain
from langchain_community.chat_models import ChatTongyi
import os

In [19]:
# 从环境变量中获取API密钥，用于初始化 ChatTongyi 模型
api_key = os.getenv("KEY_TONGYI")
if not api_key:
    raise ValueError("API Key is not set. Please ensure that the 'KEY_TONGYI' environment variable is set.")

# 初始化 ChatTongyi 模型，设置文本生成的温度参数，温度越低生成的文本越接近输入
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0,  # 设置生成文本的倾向，值越小生成的文本越接近输入
    streaming=True
)

In [34]:
# 定义第一个chain
# 提示词模板
task_description_template = PromptTemplate(
  input_variables=["poem_theme"],
  template="根据{poem_theme}这个主题写一首诗。",
)
generate_poem_chain = LLMChain(llm=llm, prompt=task_description_template, output_key="poem_content")

In [51]:
# 定义第二个chain
# 提示词模板
subtask_description_template = PromptTemplate(
  input_variables=["poem_content"],
  template="根据{poem_content}这首诗,写一个评价，并进行适当的修改。",
)
evaluate_poem_chain = LLMChain(llm=llm, prompt=subtask_description_template, output_key="poem_evaluate_content")

In [52]:
# 将两个链连接成一个顺序链
sequential_chain = SequentialChain(
    chains=[generate_poem_chain, evaluate_poem_chain],
    input_variables=["poem_theme"],  # 初始输入变量
    output_variables=["poem_content", "poem_evaluate_content"],  # 最终输出的变量
    return_all=True  # 返回所有链的输出
)

In [53]:
# 将所有链组合成一个顺序链
simple_sequential_chain = SimpleSequentialChain(
    chains=[generate_poem_chain, evaluate_poem_chain],
    input_key="poem_theme",  # 初始输入键
    output_key="poem_evaluate_content",  # 最终输出键
)


In [54]:
# 输入任务名称
input_data = {"poem_theme": "祝女朋友生日快乐"}

# 执行链
simple_sequential_chain_output = simple_sequential_chain(input_data)

In [55]:

# 输出结果
print("poem_evaluate_content:", simple_sequential_chain_output["poem_evaluate_content"])

poem_evaluate_content: 这首诗充满了深情与诗意，巧妙地融合了对生日庆祝的喜悦和对未来的美好祝愿，将对朋友或亲人的深厚情感娓娓道来。诗中运用了丰富的比喻和象征手法，如“如同晨曦中第一缕光”、“眼中的星辰大海”等，既形象生动又富有意境，能够触动人心，让人感受到作者深切的情感。

以下是对这首诗的评价及适当修改建议：

**评价：**
- **情感表达**：情感饱满，通过描绘生日庆典的温馨场景和对未来的美好期待，成功地传达了对生日主人公的关爱与祝福。
- **语言风格**：语言优美、流畅，运用了大量富有想象力的比喻，使得诗歌具有很强的艺术感染力。
- **结构布局**：整首诗结构清晰，情感递进，从对生日的庆祝到对未来生活的美好祝愿，形成了一种自然流畅的情感流动。

**修改建议**：

1. **细节描绘**：在描述生日烛光时，可以加入更多的感官细节，比如“烛光轻轻摇曳，仿佛在轻声细语”，这样的细节可以让画面更加生动立体。

2. **情感深化**：“愿你的每一天，都有爱与被爱”，这句话可以进一步深化，比如加上“愿你的世界，因爱而更加温暖，因被爱而更加坚强”，这样能更深层次地表达出对对方的关怀和期望。

3. **语言调整**：“愿你的故事，永远充满希望和勇气”这一句可以稍微调整为“愿你的每一步，都踏着希望的节拍，勇往直前”，这样不仅保留了原句的美好寓意，还增添了一种行动的力量感。

4. **结尾升华**：最后两段可以合并并稍作调整，使其更加紧凑有力，比如：“让我们携手步入这新的旅程，无论风雨晴天，彼此相依，因为有你，每一天都值得庆祝。生日快乐，愿你的梦想如繁星般璀璨，愿你的生活，永远洋溢着阳光与爱。”

通过上述修改，可以使诗歌在保持原有情感深度的同时，语言更加精炼，情感表达更为丰富，整体上更加完美和谐。


In [56]:
# 输入任务名称
input_data = {"poem_theme": "祝女朋友生日快乐"}

# 执行链
sequential_chain_output = sequential_chain(input_data)

In [57]:

# 输出结果
print("poem_content:", sequential_chain_output["poem_content"])
print("poem_evaluate_content:", sequential_chain_output["poem_evaluate_content"])


poem_content: 在时光的温柔里，你轻轻降临，
如同晨曦中第一缕光，照亮我的世界。
每一年的轮回，都成为庆祝的理由，
今天，特别的你，迎来了又一岁的绽放。

生日的烛光，轻轻摇曳，
映照出你眼中的星辰大海，
每一个愿望，都在这光亮中许下，
愿你的笑容，永远如初夏的阳光明媚。

在这个特别的日子里，我想对你说：
亲爱的，愿你的生命，如诗般细腻悠长，
每一行都是我们共同书写的美好，
每一个梦，都能在现实中轻轻触碰。

愿你的每一天，都有爱与被爱，
愿你的世界，充满色彩和奇迹，
愿你的笑声，能穿越四季，温暖每个角落，
愿你的故事，永远充满希望和勇气。

让我们一起，手牵手，步入这新的旅程，
无论是风雨还是晴天，都紧紧相依，
因为有你在身边，每一天都值得庆祝，
生日快乐，我亲爱的，愿你的世界永远璀璨。

在这特别的一刻，让我用最真挚的心语，
向你表达最深的祝福：
生日快乐，愿你的梦想如花盛开，
愿你的生活，每一天都充满阳光和爱。
poem_evaluate_content: 这首诗充满了深情与诗意，巧妙地融合了对生日庆祝的喜悦和对未来的美好祝愿，将对朋友或亲人的深厚情感娓娓道来。诗中运用了丰富的比喻和象征手法，如“如同晨曦中第一缕光”、“眼中的星辰大海”等，既形象生动又富有意境，能够触动人心，让人感受到作者深切的情感。

以下是对这首诗的评价及适当修改建议：

**评价：**
- **情感表达**：情感饱满，通过描绘生日庆典的温馨场景和对未来的美好期待，成功地传达了对生日主人公的关爱与祝福。
- **语言风格**：语言优美、流畅，运用了大量富有想象力的比喻，使得诗歌具有很强的艺术感染力。
- **结构布局**：整首诗结构清晰，情感递进，从对生日的庆祝到对未来生活的美好祝愿，形成了一种自然流畅的情感流动。

**修改建议**：

1. **细节描绘**：在描述生日烛光时，可以加入更多的感官细节，比如“烛光轻轻摇曳，仿佛在轻声细语”，这样的细节可以让画面更加生动立体。

2. **情感深化**：“愿你的每一天，都有爱与被爱”，这句话可以进一步深化，比如加上“愿你的世界，因爱而更加温暖，因被爱而更加坚强”，这样能更深层次地表达出对对方的关怀和期望。

3. **语言调整**：“愿你的故事，永远充满希望和勇气”这一句可以稍微调整为“愿你的每一步，都踏着希望的节拍，勇往直前

In [23]:
# from langchain_community.chat_models import ChatTongyi
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# import os
# # 从环境变量中获取API密钥，用于初始化 ChatTongyi 模型
# api_key = os.getenv("KEY_TONGYI")
# if not api_key:
#     raise ValueError("API Key is not set. Please ensure that the 'KEY_TONGYI' environment variable is set.")

# # 初始化 ChatTongyi 模型，设置文本生成的温度参数，温度越低生成的文本越接近输入
# llm = ChatTongyi(
#     dashscope_api_key=api_key,
#     temperature=0,  # 设置生成文本的倾向，值越小生成的文本越接近输入
#     streaming=True
# )

# prompt_template = "帮我给{product}想三个可以注册的域名"

# llm_chain = LLMChain(
#     llm=llm, 
#     prompt=PromptTemplate.from_template(prompt_template),
#     verbose=True, #是否开启日志
# )

# # llm_chain("C语言编程大师")
# llm_chain({"product": "C语言编程大师"})

e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
帮我给C语言编程大师想三个可以注册的域名

> Finished chain.


{'product': 'C语言编程大师',
 'text': '为一个专注于C语言编程的大师创建的个人品牌或网站，选择域名时应考虑与C语言、编程、技术分享等相关的关键词。以下是我为您提出的三个建议：\n\n1. **CCodeMaster.com** - 这个域名直接将"C语言"（CCode）与"大师"（Master）结合，简洁明了地传达了主题，适合一个致力于分享C语言知识和经验的专业人士。\n\n2. **TechCraftsman.com** - "Tech"代表技术，"Craftsman"则暗示了在特定领域内有深厚技艺的人士，整体域名给人一种专业且精炼的印象，适合那些在C语言编程领域有着独特见解和丰富经验的专家。\n\n3. **CCodeSage.com** - "Sage"意味着智者或有深度的知识者，与"C语言"相结合，强调了这个域名背后的人物是C语言领域的资深专家或导师。这个域名既简洁又富有深意，适合希望以智慧和经验分享知识的个人或团队。\n\n在选择域名时，请确保进行域名查询，确认所选域名未被占用，并考虑其可读性、记忆性以及SEO优化等因素。此外，保护版权和知识产权也很重要，确保域名与您的品牌或内容相符，并可能需要进行商标查询以避免潜在的法律纠纷。'}